In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time

Chrome_options = Options()
Chrome_options.add_argument("--verbose")
Chrome_options.add_argument('--no-sandbox')
Chrome_options.add_argument("--headless=new")
Chrome_options.add_argument("--disable-gpu")
Chrome_options.add_argument("--windows-size=1920,1200")
Chrome_options.add_argument("--disable-dev-shm-usage")
Chrome_options.add_experimental_option("detach", True)

driver = webdriver.Chrome(options=Chrome_options)

In [2]:
def safe_find_element(driver, by, value):
    try:
        return driver.find_element(by, value)
    except NoSuchElementException:
        return None

def news_scraping(news_url, driver):
    # 언론사
    press_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[1]/a/img[2]')
    press = press_element.get_attribute('title') if press_element else ""

    # 기사 제목
    title_element = safe_find_element(driver, By.ID, 'title_area')
    title = title_element.text if title_element else ""

    # 발행일자
    date_time_element = safe_find_element(driver, By.XPATH, '//*[@id="ct"]/div[1]/div[3]/div[1]/div/span')
    date_time = date_time_element.text if date_time_element else ""

    # 기자
    repoter_element = safe_find_element(driver, By.XPATH, '//*[@id="JOURNALIST_CARD_LIST"]/div[1]/div/div[1]/div/div/div[1]/a[2]/span/span/em')
    repoter = repoter_element.text if repoter_element else ""

    # 기사 본문
    article_element = safe_find_element(driver, By.ID, 'dic_area')
    article = article_element.text.replace("\n", "").replace("\t", "") if article_element else ""

    # 기사 반응: 쏠쏠정보
    useful_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[1]/a/span[2]')
    useful = useful_element.text if useful_element else ""

    # 기사 반응: 흥미진진
    wow_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[2]/a/span[2]')
    wow = wow_element.text if wow_element else ""

    # 기사 반응: 공감백배
    touched_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[3]/a/span[2]')
    touched = touched_element.text if touched_element else ""

    # 기사 반응: 분석탁월
    analytical_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[4]/a/span[2]')
    analytical = analytical_element.text if analytical_element else ""

    # 기사 반응: 후속강추
    recommend_element = safe_find_element(driver, By.XPATH, '//*[@id="likeItCountViewDiv"]/ul/li[5]/a/span[2]')
    recommend = recommend_element.text if recommend_element else ""

    print("뉴스:", [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url])

    return [title, press, date_time, repoter, article, useful, wow, touched, analytical, recommend, news_url]

def scraping(list_url):
    driver.implicitly_wait(3)

    news_idx = 1
    news_df = pd.DataFrame(columns = ("Title", "Press", "DateTime", "Repoter", "Article", "Useful", "Wow", "Touched", "Analytical", "Recommend", "URL"))

    for url in list_url:
        driver.get(url)
        news_df.loc[news_idx] = news_scraping(url, driver)
        news_idx += 1

    driver.close()

    return news_df

In [3]:
def make_pg_num(num):
    """Calculate the page number in the format required by the website."""
    return num if num == 1 else num+9*(num-1)

def create_url(search, page_num):
    """Create a URL with the search term and page number."""
    return f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={search}&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start={page_num}"

def make_urls(search, start_pg, end_pg):
    """Generate the URLs for the range of pages."""
    return [create_url(search, make_pg_num(i)) for i in range(start_pg, end_pg+1)]

def input_with_validation(prompt):
    """Ask for input with the given prompt, repeating until a valid integer is provided."""
    while True:
        try:
            return int(input(prompt))
        except ValueError:
            print("Invalid input, please enter an integer.")

def main():
    search = input("검색 키워드를 입력해주세요: ")

    start_pg = input_with_validation("\n크롤링 시작 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 시작 페이지: {start_pg}페이지")

    end_pg = input_with_validation("\n크롤링 종료 페이지를 입력해주세요. ex)1(숫자만 입력): ")
    print(f"\n크롤링 종료 페이지: {end_pg}페이지")

    return make_urls(search, start_pg, end_pg)

if __name__ == "__main__":
    search_urls = main()
    print("생성된 URL: ", search_urls)


크롤링 시작 페이지: 1페이지

크롤링 종료 페이지: 2페이지
생성된 URL:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경기도지사 김동연&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=경기도지사 김동연&sort=0&photo=0&field=0&pd=0&ds=&de=&cluster_rank=17&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:all,a:all&start=11']


In [4]:
# Initialize the list to store the links
list_url = []

# Iterate over the URLs
for url in search_urls:
    # Send GET request to the web page
    response = requests.get(url)

    # If the request is successful, extract the HTML content and create a BeautifulSoup object
    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        links = soup.select("a.info, a.sub_txt")

        # Filter and save the links with "naver.com" in their address
        for link in links:
            href = link.get("href")
            if "naver.com" in href:
                list_url.append(href)
    else:
        print("The request failed.")

    # Sleep for 1 second
    time.sleep(2)

In [5]:
list_url

['https://n.news.naver.com/mnews/article/003/0011939025?sid=102',
 'https://n.news.naver.com/mnews/article/421/0006893842?sid=102',
 'https://n.news.naver.com/mnews/article/003/0011940877?sid=102',
 'https://n.news.naver.com/mnews/article/011/0004207399?sid=102',
 'https://n.news.naver.com/mnews/article/014/0005033714?sid=102',
 'https://n.news.naver.com/mnews/article/031/0000755082?sid=004',
 'https://n.news.naver.com/mnews/article/020/0003506271?sid=100',
 'https://n.news.naver.com/mnews/article/016/0002161731?sid=100',
 'https://n.news.naver.com/mnews/article/119/0002725274?sid=102',
 'https://n.news.naver.com/mnews/article/003/0011938261?sid=102',
 'https://n.news.naver.com/mnews/article/001/0014029209?sid=105',
 'https://n.news.naver.com/mnews/article/421/0006892001?sid=101',
 'https://n.news.naver.com/mnews/article/003/0011938745?sid=102',
 'https://n.news.naver.com/mnews/article/421/0006892006?sid=101',
 'https://n.news.naver.com/mnews/article/001/0014029213?sid=101',
 'https://

In [6]:
scraping(list_url)

뉴스: ['김동연 "팹리스 집적단지 조성해 용인 반도체 성공 힘 보탤 것"', '뉴시스', '2023.06.27. 오후 6:24', '박상욱 기자', '시스템반도체 경쟁력 확보, 제3판교 조성 계획 밝혀기반 시설 기초지자체 간 갈등 조정인력 양성 및 조달 등 경기도 역할 강조김동연 경기도지사가 27일 오후 삼성전자 기흥캠퍼스에서 열린 용인 첨단 시스템반도체 성공을 위한 회의에 참석했다. (사진=경기도 제공) *재판매 및 DB 금지[수원=뉴시스] 박상욱 기자 = 김동연 경기도지사가 제3판교테크노밸리에 시스템반도체 경쟁력 확보를 위한 팹리스 집적단지를 조성해 용인 첨단시스템반도체 국가산단의 성공적인 추진에 힘을 보태겠다고 강조했다.김동연 지사는 27일 용인 삼성전자 기흥캠퍼스에서 \'용인 국가산단 성공 추진을 위한 상생협약\'을 체결한 뒤 기자들과 만나 이같이 말했다.김 지사는 "우리나라 시스템반도체에서 부족한 부분이 팹리스다. 경기도가 제3판교테크노밸리를 중심으로 팹리스 집적단지를 조성할 계획이다. 그렇게 되면 시스템반도체의 경쟁력 확보에 시너지 효과를 낼 수 있다"라면서 "그곳에 팹리스와 관련된 벤처나 스타트업 기업을 유치해 반도체 산업 생태계를 조성할 계획이다. 3판교 이외에도 경기도 여러 곳에서 이런 팹리스 및 전후방 산업 집적단지가 만들어질 수 있도록 노력할 방침"이라고 말했다.도는 국내 팹리스 기업 144개 사 가운데 51%가 경기도에 위치하고 있어 제3판교테크노밸리를 중심으로 팹리스 산업을 집중 유치하면 용인 첨단시스템반도체 국가산단과 시너지 효과를 얻을 것으로 기대하고 있다.김 지사는 이 밖에도 원활한 국가산단 조성을 위해 용수나 전력 등 기반 시설 설치 시 최선을 다해 기초자치단체 간 현안 조정 역할을 하겠다는 뜻도 밝혔다. 인력 양성도 적극 지원하겠다고 강조했다.그는 "인력 양성은 반도체 투자 관련해서 가장 중요하고 힘든 부분"이라며 "경기도가 여러 가지 대학과 또 공공기관, 산학과 같이하는 인력 양성 프로그램을 만들어서 인력 조달에 충분한 역할

,Title,Press,DateTime,Repoter,Article,Useful,Wow,Touched,Analytical,Recommend,URL
1,"김동연 ""팹리스 집적단지 조성해 용인 반도체 성공 힘 보탤 것""",뉴시스,2023.06.27. 오후 6:24,박상욱 기자,"시스템반도체 경쟁력 확보, 제3판교 조성 계획 밝혀기반 시설 기초지자체 간 갈등 조...",1,0,1,0,0,https://n.news.naver.com/mnews/article/003/001...
2,본격 출발하는 김동연의 ‘기회소득·국제공항·경기북도’,뉴스1,2023.06.28. 오후 12:52,송용환 기자,"도의회, 3개 핵심안 등 총 65개 안건 의결 후 폐회‘기회소득’ 등 김동연 경기도...",0,0,0,0,0,https://n.news.naver.com/mnews/article/421/000...
3,"김동연표 '국제공항·기회소득' 조례, 우여곡절 끝 도의회 통과",뉴시스,2023.06.28. 오후 2:10,이병희 기자,"경기도의회, 제369회 정례회 4차 본회의…65건 처리이애형 의원, '경기국제공항 ...",0,0,0,0,0,https://n.news.naver.com/mnews/article/003/001...
4,'경기국제공항사업' 등 김동연 핵심공약 도의회 문턱 넘어,서울경제,2023.06.28. 오후 3:37,,경기북부특별자치도 특위·예술인·장애인 기회소득 근거도 마련5월31일 오전 경기도청 ...,0,0,0,0,0,https://n.news.naver.com/mnews/article/011/000...
5,김동연표 '기회소득' 7월 지급 확정...조례안 통과,파이낸셜뉴스,2023.06.28. 오후 1:40,장충식 기자,기회소득 지급 위한 대상자 신청 공모 등 절차 추진【파이낸셜뉴스 수원=장충식 기자】...,0,0,0,0,0,https://n.news.naver.com/mnews/article/014/000...
6,김동연표 기회소득...예술인·장애인 7월 지급,아이뉴스24,2023.06.28. 오후 2:22,김아라 기자,김동연 경기도지사의 역점사업인 ‘예술인·장애인 기회소득’을 내달부터 지급한다.경기도...,0,0,0,0,0,https://n.news.naver.com/mnews/article/031/000...
7,김동연은 ‘경기지사 징크스’를 끊어낼 수 있을까[한상준의 정치 인사이드],동아일보,2023.06.28. 오후 12:01,,김동연 경기도지사가 8일 오전 경기 수원시 경기도청 경기도지사실에서 동아일보와 인터...,154,2,8,1,3,https://n.news.naver.com/mnews/article/020/000...
8,"이재명 전 경기지사 체육시설 운영권, 김동연 ‘원대 복귀’ 시켰다",헤럴드경제,2023.06.27. 오후 2:05,박정규 기자,"“이재명, 경기도 체육시설 운영권 GH 준것은 잘못 “이라는 의미[경기도 제공][헤...",0,1,1,0,0,https://n.news.naver.com/mnews/article/016/000...
9,"경기도-도의회, 체육 분야 위탁사무 이관 협력방안 논의",데일리안,2023.06.27. 오후 3:15,윤종열 기자,도 체육 사무 위탁 관련 당사자 간 협의 통해 안정적 이관 방안 마련키로ⓒ[데일리안...,0,0,0,0,0,https://n.news.naver.com/mnews/article/119/000...
10,"경기도·도의회, 운동부·체육시설 체육단체 이관 뜻 모아",뉴시스,2023.06.27. 오후 3:06,박상욱 기자,관련 기관 협의 통해 안정적 방안 마련체육 분야 위탁사무 이관 관련 간담회 *재판매...,0,0,0,0,0,https://n.news.naver.com/mnews/article/003/001...


In [7]:
import os
import openai
import pandas as pd
from docx import Document

# Assuming that you have the DataFrame 'news_df' already loaded
def ask_from_article(index):
    article = news_df['Article'][index]

    # Load API key from file
    with open("key.txt", "r") as f:
        api_key = f.read().strip()

    openai.api_key = api_key

    # Role assignment (competent journalist and text analytics expert)
    messages = [
        {"role": "system", "content": "You are a very competent journalist and text analytics expert who needs to do the following."},
        {"role": "user", "content": f"Here is an article: {article}"}
    ]

    while True:
        user_content = input("기사에 대한 질문을 입력하세요. : ")
        if user_content.lower() == "종료":
            break
        messages.append({"role": "user", "content": f"{user_content}"})

        completion = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=messages
        )

        assistant_content = completion.choices[0].message["content"].strip()

        messages.append({"role": "assistant", "content": f"{assistant_content}"})

        print(f"GPT-3.5 Turbo: {assistant_content}")

    # Saving the conversation to a Word file
    doc = Document()
    for message in messages:
        doc.add_paragraph(f"{message['role']}: {message['content']}")
    doc.save("대화기록.docx")
